In [1]:
import os
import yaml

os.chdir('Z:/Covid19/Covid19_OMOP/new_pipeline/')
print(os.getcwd())

import pandas as pd
from omop_etl.load import Loader
# from .omop_etl.stage import STAGE

from pydma.utils import find

Z:\Covid19\Covid19_OMOP\new_pipeline


In [2]:
omop = Loader('config.yml')
STAGE = omop.stage

# Row counts

In [3]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [4]:
loaded = omop.config.load

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                count = omop.row_count(stg_name, schema='stage')
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            count = omop.row_count(stg_name, schema='stage')
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Stage_RC'])
# table_counts.head(50)

In [5]:
# table_counts

In [6]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['Preload_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='preload') if t not in ('death','person','visit_occurrence') else 0)
count_diff['Load_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['DeId_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))

In [7]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()

In [8]:
print(count_diff)

                  Table   Stage_RC  Preload_RC    Load_RC    DeId_RC
0  condition_occurrence   67002743    69174425   56801747   56801747
1                 death       6301           0       6301       6301
2         drug_exposure   73551749    73444222   73408463   73408463
3           measurement  357397202   412434410  422814217  422814941
4           observation   71381467    71486920   87570857   87570857
5                person     204936           0     204935     204935
6  procedure_occurrence   26568160    26462556   12009481   12009481
7      visit_occurrence   23734186           0   23656621   23656621


# Count concept ids

# Date range